In [1]:
library(Signac)
library(Seurat)
library(GenomicRanges)
library(BSgenome.Mmusculus.UCSC.mm10)
library(EnsDb.Mmusculus.v79)
library(Matrix)

annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
seqlevelsStyle(annotations) <- "UCSC"
genome(annotations) <- "mm10"

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int,

In [2]:
# # load metadata from original paper
# metadata <- read.table("/mnt/disk1/xiaojk/data/hippocampus/Supplementary_Table_2-Metatable_of_nuclei.tsv", sep="\t", skip=1)
# rownames(metadata) <- metadata$V1
# colnames(metadata) <- c("cell", "sample", "barcode", "logUM", "TSSe", "class", "MajorType", "SubType", "na")
# cells <- metadata$cell

In [3]:
# download from http://catlas.org/mousebrain/#!/cellBrowser
# metadata1 <- read.table("/mnt/disk1/xiaojk/data/hippocampus/NonN_meta.tsv", sep="\t", header=TRUE)  #, skip=1
# metadata2 <- read.table("/mnt/disk1/xiaojk/data/hippocampus/Glutamate_meta.tsv", sep="\t", header=TRUE)  #, skip=1
# metadata3 <- read.table("/mnt/disk1/xiaojk/data/hippocampus/GABA_meta.tsv", sep="\t", header=TRUE)  #, skip=1
# metadata <- rbind(metadata1, metadata2, metadata3)
# write.table(metadata, "/mnt/disk1/xiaojk/data/hippocampus/all_meta.tsv", sep="\t")

In [4]:
metadata <- read.table("/mnt/disk1/xiaojk/data/hippocampus/all_meta.tsv", sep="\t", header=TRUE, row.names = 1)  #, skip=1
metadata_sub <- metadata[metadata$DissectionRegion %in% c('8J','8E') & metadata$replicate %in% c(1), ]
cells <- metadata_sub$cellID
length(metadata_sub)

[1] 11

In [22]:
metadata_sub

,cellID,sample,replicate,logUMI,tsse,DissectionRegion,RegionName,MajorRegion,SubRegion,Slice,SubClass
,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
.AAACTACCAGAAATTGAGGAGG,CEMBA190711_8E.AAACTACCAGAAATTGAGGAGG,CEMBA190711_8E,1,3.201943,22.07792,8E,CA-1,HPF,CA,slice: 8,ASC
.AAACTACCAGAACCCACTATCT,CEMBA190711_8E.AAACTACCAGAACCCACTATCT,CEMBA190711_8E,1,3.595717,24.71591,8E,CA-1,HPF,CA,slice: 8,OPC
.AAACTACCAGGAAGGGTGGTAT,CEMBA190711_8E.AAACTACCAGGAAGGGTGGTAT,CEMBA190711_8E,1,3.289143,23.52941,8E,CA-1,HPF,CA,slice: 8,ASC
.AAACTACCAGGCGAACGTAGAC,CEMBA190711_8E.AAACTACCAGGCGAACGTAGAC,CEMBA190711_8E,1,3.349472,26.34508,8E,CA-1,HPF,CA,slice: 8,ASC
.AAACTACCAGGGCCTCTGCAAT,CEMBA190711_8E.AAACTACCAGGGCCTCTGCAAT,CEMBA190711_8E,1,3.385070,23.86364,8E,CA-1,HPF,CA,slice: 8,OGC
.AACTGCGCCAAAGTGTCGTGGA,CEMBA190711_8E.AACTGCGCCAAAGTGTCGTGGA,CEMBA190711_8E,1,3.073718,32.00000,8E,CA-1,HPF,CA,slice: 8,ASC
.AACTGCGCCACTTGAGACTCCG,CEMBA190711_8E.AACTGCGCCACTTGAGACTCCG,CEMBA190711_8E,1,3.080987,23.77622,8E,CA-1,HPF,CA,slice: 8,OGC
.AACTTCTGCTGGGAACAAGTCA,CEMBA190711_8E.AACTTCTGCTGGGAACAAGTCA,CEMBA190711_8E,1,3.053463,21.59091,8E,CA-1,HPF,CA,slice: 8,ASC
.AACTTCTGCTTGCGAAGCTCAC,CEMBA190711_8E.AACTTCTGCTTGCGAAGCTCAC,CEMBA190711_8E,1,3.171726,20.29598,8E,CA-1,HPF,CA,slice: 8,ASC


In [5]:
length(cells)

[1] 16270

In [6]:
frags <- CreateFragmentObject(
  path = "/mnt/disk1/xiaojk/data/hippocampus/fragments.sort.bed.gz",
  cells = cells,
  validate.fragments = FALSE
)

Computing hash



In [25]:
frags

A Fragment object for 16270 cells

In [8]:
attr(frags@cells, ".match.hash") <- NULL

In [9]:
unified.peaks <- read.table("/mnt/disk1/xiaojk/data/hippocampus/unified_peaks.bed", sep = "\t", header = TRUE)
unified.peaks <- makeGRangesFromDataFrame(unified.peaks)

In [10]:
unified.peaks

GRanges object with 88333 ranges and 0 metadata columns:
          seqnames          ranges strand
             <Rle>       <IRanges>  <Rle>
      [1]     chr1 3012371-3012843      *
      [2]     chr1 3060652-3061121      *
      [3]     chr1 3094790-3095399      *
      [4]     chr1 3113044-3113920      *
      [5]     chr1 3119264-3120628      *
      ...      ...             ...    ...
  [88329]     chrY   872791-873259      *
  [88330]     chrY   896966-898147      *
  [88331]     chrY 1009367-1010739      *
  [88332]     chrY 1114059-1114362      *
  [88333]     chrY 1244820-1246026      *
  -------
  seqinfo: 21 sequences from an unspecified genome; no seqlengths

In [11]:
frags

A Fragment object for 16270 cells

In [24]:
# quantify  这里是关键，统计生成peak矩阵
counts <- FeatureMatrix(
  fragments = frags,
  features = unified.peaks,
#   cells = cells,
  process_n = 2000
)
#   cells = cells

Extracting reads overlapping genomic regions



In [26]:
dim(counts)

[1] 88333 16270

In [14]:
comm_cell <- intersect(cells, colnames(counts)) ## 取交集

In [15]:
length(unique(comm_cell))

[1] 16270

In [16]:
dim(metadata_sub)

[1] 16270    11

In [17]:
counts_sub <- counts[,comm_cell] 

In [18]:
dim(counts_sub)

[1] 88333 16270

In [19]:
rownames(metadata_sub) <- substring(metadata_sub$cellID, 15)
csums <- colSums(counts_sub)
rsums <- rowSums(counts_sub)
counts_sub <- counts_sub[rsums > 100, csums > 100]

In [20]:
counts_sub

0 x 0 sparse Matrix of class "ngCMatrix"
<0 x 0 matrix>

In [21]:
pbmc <- CreateSeuratObject(
  counts = counts_sub,
  fragments = frags,
  genome = "mm10",
  annotation = annotations
)

biccn <- CreateSeuratObject(
  counts = biccn_assay,
  assay = "ATAC",
  project = "BICCN",
  meta.data = metadata_sub
)

ERROR: Error in CreateAssayObject(counts = counts, min.cells = min.cells, min.features = min.features, : No cell names (colnames) names present in the input matrix


In [ ]:
pbmc

In [ ]:
granges(pbmc)

In [ ]:
# extract gene annotations from EnsDb
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)

# change to UCSC style since the data was mapped to hg19
seqlevels(annotations) <- paste0('chr', seqlevels(annotations))
genome(annotations) <- "mm10"

In [ ]:
# add the gene information to the object
Annotation(pbmc) <- annotations

In [ ]:
annotations

In [ ]:
# compute nucleosome signal score per cell
pbmc <- NucleosomeSignal(object = pbmc)

# compute TSS enrichment score per cell
pbmc <- TSSEnrichment(object = pbmc, fast = FALSE)

In [ ]:
pbmc$blacklist_fraction <- FractionCountsInRegion(
  object = pbmc,
  assay = 'peaks',
  regions = blacklist_mm10
)

In [ ]:
# add blacklist ratio and fraction of reads in peaks
# pbmc$pct_reads_in_peaks <- pbmc$peak_region_fragments / pbmc$passed_filters * 100
# pbmc$blacklist_ratio <- pbmc$blacklist_region_fragments / pbmc$peak_region_fragments

In [ ]:
DensityScatter(pbmc, x = 'nCount_peaks', y = 'TSS.enrichment', log_x = TRUE, quantiles = TRUE)

In [ ]:
pbmc$high.tss <- ifelse(pbmc$TSS.enrichment > 3, 'High', 'Low')
TSSPlot(pbmc, group.by = 'high.tss') + NoLegend()

In [ ]:
# pbmc$nucleosome_group <- ifelse(pbmc$nucleosome_signal > 4, 'NS > 4', 'NS < 4')
# FragmentHistogram(object = pbmc, group.by = 'nucleosome_group')

In [ ]:
VlnPlot(
  object = pbmc,
  features = c('nCount_peaks', 'TSS.enrichment', 'nucleosome_signal', 'pct_reads_in_peaks'),
  pt.size = 0.1,
  ncol = 3
)

In [ ]:
pbmc <- subset(
  x = pbmc,
  subset = blacklist_fraction < 0.03 &
    TSS.enrichment < 20 
)
pbmc

In [ ]:
pbmc <- RunTFIDF(pbmc)
pbmc <- FindTopFeatures(pbmc, min.cutoff = 'q0')
pbmc <- RunSVD(pbmc)

In [ ]:
DepthCor(pbmc)

In [ ]:
pbmc <- RunUMAP(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindNeighbors(object = pbmc, reduction = 'lsi', dims = 2:30)
pbmc <- FindClusters(object = pbmc, verbose = FALSE, algorithm = 3)
DimPlot(object = pbmc, label = TRUE) + NoLegend()

In [ ]:
gene.activities <- GeneActivity(pbmc)

In [ ]:
pbmc[['RNA']] <- CreateAssayObject(counts = gene.activities)
pbmc <- NormalizeData(
  object = pbmc,
  assay = 'RNA',
  normalization.method = 'LogNormalize',
  scale.factor = median(pbmc$nCount_RNA)
)

In [ ]:
DefaultAssay(pbmc) <- 'RNA'

In [ ]:
rna <- pbmc[['RNA']]

In [ ]:
seurat_obj <- CreateSeuratObject(counts = pbmc[['RNA']])

In [ ]:
seurat_obj[["RNA"]] <- pbmc[['RNA']]

In [ ]:
seurat_obj